In [1]:
import os
import sys

import numpy as np
import torch
import torch.nn as nn
from torch.nn.utils.rnn import pad_sequence
from tqdm import tqdm

parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.insert(0, parent_dir)

from dataloader import FastTensorDataLoader

In [2]:
np.random.seed(0)
torch.manual_seed(0)

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### Synthetic Data Generation

In [4]:
def generate_poisson_points(kappa, scale, region):
    """
    Generate a Poisson Point Process in a 2D region based on intensity function.
    
    Parameters:
    - kappa (torch.Tensor): The intensity parameter (scalar or vector).
    - scale (torch.Tensor): The scale parameter (scalar or vector).
    - region (tuple): The spatial domain as ((xmin, xmax), (ymin, ymax)).
    
    Returns:
    - points (numpy.ndarray): The simulated points of the PPP.
    """
    (xmin, xmax), (ymin, ymax) = region

    area = (xmax - xmin) * (ymax - ymin)
    max_intensity = kappa * area  # Maximum value of intensity
    num_samples = np.random.poisson(lam=max_intensity)

    x_candidates = np.random.uniform(xmin, xmax, size=num_samples)
    y_candidates = np.random.uniform(ymin, ymax, size=num_samples)
    candidates = torch.tensor(np.stack([x_candidates, y_candidates], axis=1), dtype=torch.float32)
    
    squared_norm = torch.sum(candidates**2, dim=-1)
    intensity = kappa * torch.exp(-squared_norm / scale**2)
    
    uniform_samples = torch.rand(num_samples)  # Uniform samples for rejection
    acceptance_mask = uniform_samples < (intensity / kappa)
    
    accepted_points = candidates[acceptance_mask]
    return accepted_points

In [5]:
true_kappa = 100.0
true_scale = 0.5
region = ((0, 1), (0, 1))

In [6]:
num_samples = 1000
samples = []
for _ in range(num_samples):
    x_t = generate_poisson_points(true_kappa, true_scale, region)
    samples.append(torch.tensor(x_t))

C:\Users\79217\AppData\Local\Temp\ipykernel_21680\2803613514.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  samples.append(torch.tensor(x_t))


In [7]:
X = pad_sequence(samples, batch_first=True, padding_value=0)
lengths = torch.tensor([len(s) for s in samples], dtype=torch.int64)

lengths_expanded = lengths.unsqueeze(-1).expand(-1, X.shape[1])
X = torch.cat((X, lengths_expanded.unsqueeze(-1)), dim=-1)

X_train = X[:100]
X_val = X[100:150]
loader_train = FastTensorDataLoader(X_train, batch_size=1000, shuffle=True)
loader_val = FastTensorDataLoader(X_val, batch_size=1000, shuffle=True)

### NLL Model

In [8]:
class InhomPoissonNLL(nn.Module):
    def __init__(self, region):
        super().__init__()
        self.region = region
        self.kappa = nn.Parameter(torch.tensor([np.abs(np.random.randn())], dtype=torch.float32, device=device))
        self.scale = nn.Parameter(torch.tensor([np.abs(np.random.randn())], dtype=torch.float32, device=device))

    def intensity(self, x, y, epsilon=1e-10):
        return self.kappa * torch.exp(-(x**2 + y**2) / self.scale**2) + epsilon

    def forward(self, points):
        nll = 0.0
        (x_min, x_max), (y_min, y_max) = self.region
        x_grid = torch.linspace(x_min, x_max, steps=100)
        y_grid = torch.linspace(y_min, y_max, steps=100)
        
        xx, yy = torch.meshgrid(x_grid, y_grid)
        partition = torch.trapz(torch.trapz(self.intensity(xx, yy), dx=0.01), dx=0.01)
        for point in points:
            length = int(point[0, -1])
            x, y = point[:length, 0], point[:length, 1]
            log_likelihood = torch.sum(torch.log(self.intensity(x, y)))
            
            # Add to the total NLL
            nll += log_likelihood - partition
        
        return -nll 

### SM Model

In [9]:
class PoissonSM(nn.Module):
    def __init__(self):
        super().__init__()
        self.scale = nn.Parameter(torch.tensor([torch.abs(torch.randn(1))], dtype=torch.float32))

    def forward(self, x):
        squared_norm = torch.sum(x**2, dim=-1)
        return - squared_norm / self.scale**2

    def compute_psi(self, x):
        x.requires_grad_()
        nn_output = self.forward(x)
        psi = torch.autograd.grad(nn_output, x, grad_outputs=torch.ones_like(nn_output), create_graph=True)[0]
        return psi

    def loss(self, points):
        lengths = points[:, 0, -1].to(dtype=torch.int64)
        max_length = lengths.max()
        x_t = points[:, :max_length, :-1]  # Pad to max length in batch
 
        psi_x = self.compute_psi(x_t)
        norm_squared = (psi_x ** 2).sum(dim=-1)  # Sum across all dimensions

        # padded values give none zero divergence -> mask 
        mask = torch.arange(max_length, device=x_t.device).unsqueeze(0) < lengths.unsqueeze(1)
        divergence = 0
        for i in range(x_t.shape[-1]):  # Iterate over the features of x
            gradient = torch.autograd.grad(psi_x[..., i].sum(), x_t, retain_graph=True, create_graph=True)[0]
            divergence += gradient[..., i]  # Sum over each feature dimension
        
        divergence = divergence * mask
        total_loss = 0.5 * norm_squared + divergence
        total_loss = total_loss.sum(dim=-1)/lengths  # Sum over the time dimension
        
        return total_loss.mean()

### Training function

In [10]:
def train_model(model, optimizer, loader_train, num_epochs=1000):
    for _ in tqdm(range(num_epochs), desc="Running epochs"):
        for X_batch in loader_train:
            optimizer.zero_grad()
            if isinstance(model, InhomPoissonNLL):
                loss = model(X_batch[0])
            else:
                loss = model.loss(X_batch[0])
            loss.backward()
            optimizer.step()
    return model

### Instantiate models

In [11]:
nll_model = InhomPoissonNLL(region).to(device)
sm_model = PoissonSM().to(device)

nll_optimizer = torch.optim.Rprop(nll_model.parameters())
sm_optimizer = torch.optim.Rprop(sm_model.parameters())

### Train models

In [12]:
trained_nll = train_model(nll_model, nll_optimizer, loader_train)

Running epochs:   0%|          | 0/1000 [00:00<?, ?it/s]

c:\Users\79217\Documents\TUD\3 semester\Research Project\poisson_inference\.venv\Lib\site-packages\torch\functional.py:534: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\TensorShape.cpp:3596.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Running epochs: 100%|██████████| 1000/1000 [00:59<00:00, 16.78it/s]


In [13]:
trained_sm = train_model(sm_model, sm_optimizer, loader_train)

Running epochs: 100%|██████████| 1000/1000 [00:06<00:00, 163.51it/s]


### Results

In [14]:
nll_results = {"kappa": trained_nll.kappa.item(), "scale": trained_nll.scale.item()}
sm_results = {"scale": trained_sm.scale.item()}


# Print Results
print("Comparison of Methods:")
print("NLL Method:")
print(f"  Estimated kappa: {nll_results['kappa']}")
print(f"  Estimated scale: {nll_results['scale']}")

print("\nSM Method:")
print(f"  Estimated scale: {sm_results['scale']}")


Comparison of Methods:
NLL Method:
  Estimated kappa: 102.14119720458984
  Estimated scale: -0.5042386651039124

SM Method:
  Estimated scale: 0.4922265410423279
